> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.6 - Pandas

## Exercícios
Antes de continuar, baixe os arquivos das bases de dados de partidas em Copas do Mundo e salve na mesma pasta deste notebook.
* https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/fifa-wc/matches_1930_2022.csv
* https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/fifa-wc/matches_1991_2023.csv

A célula a seguir já carrega os dados em CSV e ajusta as colunas para trabalharmos com os nomes traduzidos (como fizemos em aula).

In [ ]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',12)

wcwomen_df = pd.read_csv('matches_1991_2023.csv')
wcmen_df   = pd.read_csv('matches_1930_2022.csv')
wc = pd.concat((wcwomen_df,wcmen_df)).reset_index()

nomes_traduzidos = {'home_team': 'time_1', 'away_team': 'time_2', 'home_score': 'gols_1', 'away_score': 'gols_2',
                    'Date': 'data', 'Year': 'ano', 'Host': 'país_sede', 'Attendance': 'comparecimento',
                    'Score': 'resultado', 'Round': 'rodada', 'home_goal': 'gols_1_detalhes', 'away_goal': 'gols_2_detalhes',
                    'home_own_goal': 'gols_1_contra', 'away_own_goal': 'gols_2_contra',
                    'home_penalty_goal': 'gols_1_penalti', 'away_penalty_goal': 'gols_2_penalti',
                    'home_red_card': 'cartao_vermelho_1', 'away_red_card': 'cartao_vermelho_2',
                    'home_yellow_card_long': 'cartao_amarelo_1', 'away_yellow_card_long': 'cartao_amarelo_2'}

wc = wc.loc[:, nomes_traduzidos.keys()]
wc.columns = nomes_traduzidos.values()

copa = wc['ano'].apply( lambda x: 'Masculina' if x % 2 == 0 else 'Feminina').astype('string')
wc['copa'] = copa
display(wc)

#### Q1.
Faça as conversões de tipo necessárias para que a saída do comando `wc.info()` seja como apresentado a seguir. E **salve o novo dataframe** com o comando `df.to_csv('wc_formatado.csv')`.

```
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   time_1             1312 non-null   string        
 1   time_2             1312 non-null   string        
 2   gols_1             1312 non-null   int64         
 3   gols_2             1312 non-null   int64         
 4   data               1312 non-null   datetime64[ns]
 5   ano                1312 non-null   int64         
 6   país_sede          1312 non-null   string        
 7   comparecimento     1312 non-null   int64         
 8   resultado          1312 non-null   string        
 9   rodada             1312 non-null   category      
 10  gols_1_detalhes    970 non-null    string        
 11  gols_2_detalhes    771 non-null    string        
 12  gols_1_contra      57 non-null     string        
 13  gols_2_contra      30 non-null     string        
 14  gols_1_penalti     170 non-null    string        
 15  gols_2_penalti     119 non-null    string        
 16  cartao_vermelho_1  59 non-null     string        
 17  cartao_vermelho_2  65 non-null     string        
 18  cartao_amarelo_1   834 non-null    string        
 19  cartao_amarelo_2   857 non-null    string        
 20  copa               1312 non-null   string 
```

In [ ]:
# 1) Colunas a garantir como string (manter None/NaN como pd.NA)
colunas_string = [
    'time_1', 'time_2', 'país_sede', 'resultado',
    'gols_1_detalhes', 'gols_2_detalhes',
    'gols_1_contra', 'gols_2_contra',
    'gols_1_penalti', 'gols_2_penalti',
    'cartao_vermelho_1', 'cartao_vermelho_2',
    'cartao_amarelo_1', 'cartao_amarelo_2', 'copa'
]
# Só converte as que existem no dataframe
colunas_string = [c for c in colunas_string if c in wc.columns]
wc[colunas_string] = wc[colunas_string].astype('string')

# 2) Colunas numéricas: gols_1, gols_2, ano, comparecimento
# Alguns datasets podem ter missing -> preencher NaN por 0 antes da conversão se necessário
for col in ['gols_1', 'gols_2', 'ano', 'comparecimento']:
    if col in wc.columns:
        # converter para numérico, coerce -> NaN, depois preencher com 0 ou converter para int se não houver NaNs
        wc[col] = pd.to_numeric(wc[col], errors='coerce')

# Se comparecimento tiver NaN, podemos preencher com 0 (ou manter NaN). Exemplo do enunciado mostra int64 não-nulos.
# Aqui preencherei NaN por 0 para seguir o exemplo (se preferir manter NA, remova a linha abaixo)
if 'comparecimento' in wc.columns:
    wc['comparecimento'] = wc['comparecimento'].fillna(0).astype('int64')

# Gols e ano também convertidos para int (preenchendo NaN por 0 caso existam)
for col in ['gols_1','gols_2','ano']:
    if col in wc.columns:
        wc[col] = wc[col].fillna(0).astype('int64')

# 3) Converter coluna 'data' para datetime
if 'data' in wc.columns:
    wc['data'] = pd.to_datetime(wc['data'], errors='coerce')

# 4) Converter 'rodada' para categoria (se existir)
if 'rodada' in wc.columns:
    wc['rodada'] = wc['rodada'].astype('category')

# Conferir informação final conforme enunciado
print("=== wc.info() após conversões ===")
wc.info()

# Salvar o dataframe formatado
wc.to_csv('wc_formatado.csv', index=False)
print("\nArquivo 'wc_formatado.csv' salvo.")

#### Q2.
Apresente a linha do dataframe `wc` que corresponde ao jogo com maior audiência da história.

In [ ]:
# Garante que comparecimento é numérico
wc['comparecimento'] = pd.to_numeric(wc['comparecimento'], errors='coerce').fillna(0).astype('int64')

# Índice da maior audiência
idx_max = wc['comparecimento'].idxmax()
jogo_mais_audiencia = wc.loc[idx_max]

print("=== Jogo com maior audiência ===")
display(jogo_mais_audiencia.to_frame().T)  # exibe como tabela de 1 linha

#### Q3.
Aplicando operações sobre as colunas `ano` e `copa` do dataframe `wc`, apresente quantas copas femininas e quantas copas masculinas já aconteceram na história.

Exemplo de saída (valores inventados):
```
Masculina: 22
Feminina: 9
```

In [ ]:
# Considera-se número de anos distintos por tipo de copa (cada 'ano' corresponde a uma edição)
copas_por_tipo = wc[['ano','copa']].drop_duplicates().groupby('copa')['ano'].nunique().to_dict()

print("Número de edições por tipo de copa:")
for tipo, n in copas_por_tipo.items():
    print(f"{tipo}: {n}")

#### Q3. 
Crie um dataframe `participacao` com as colunas `país`, `copa`, e `num_copas` que armazena a quantidade de copas do mundo que cada país participou, informando se é da copa masculina ou feminina. Ao final imprima, como apresentado a seguir, o top 5 países de cada competição que mais participou de copas do mundo.

Exemplo de saída (valores inventados):
```
país            copa        num_copas
Brazil          Feminina    8
Unites States   Feminina    8
Germany         Feminina    8
Japan           Feminina    7
Colombia        Feminina    7
```

```
país            copa        num_copas
Brazil          Masculina   21
Germany         Masculina   21
Argentina       Masculina   20
England         Masculina   20
Mexico          Masculina   20
```

In [ ]:
# 1) Criar um DataFrame longo com colunas: 'país', 'ano', 'copa' (cada participação de um time em uma edição)
# Para isso, vamos transformar linhas com time_1/time_2 em formato long

# selecionar colunas essenciais (algumas podem não existir dependendo do CSV)
cols_needed = ['time_1','time_2','ano','copa']
for c in cols_needed:
    if c not in wc.columns:
        raise RuntimeError(f"Coluna esperada '{c}' não encontrada no dataframe.")

# times casa
df_casa = wc[['time_1','ano','copa']].rename(columns={'time_1':'país'})
# times fora
df_fora = wc[['time_2','ano','copa']].rename(columns={'time_2':'país'})

# concat e remover duplicados país-ano (um país pode aparecer várias vezes em mesma edição)
df_part = pd.concat([df_casa, df_fora], ignore_index=True).drop_duplicates(subset=['país','ano','copa'])

# agora agrupa por país e copa e conta anos distintos (num_copas)
participacao = df_part.groupby(['país','copa'])['ano'].nunique().reset_index().rename(columns={'ano':'num_copas'})

# imprimir top5 para cada copa (Feminina e Masculina)
print("=== Top 5 países com mais participações (Feminina) ===")
top_fem = participacao[participacao['copa']=='Feminina'].sort_values('num_copas', ascending=False).head(5)
display(top_fem)

print("\n=== Top 5 países com mais participações (Masculina) ===")
top_masc = participacao[participacao['copa']=='Masculina'].sort_values('num_copas', ascending=False).head(5)
display(top_masc)

#### Q4. 
* Crie um dataframe `gols` com duas colunas `país` e `total_gols` com o total de gols marcados por cada país durante todas as copas do mundo, juntando gols em casa (`gols_1`) e gols como time visitante (`gols_2`).
* Imprima o dataframe `gols` ordenado de forma decrescente, para que os times com mais gols fiquem no topo.

Segue um exemplo ilustrativo com o formato do dataframe resultado:

```
país        total_gols
Brazil      120
Argentina   112
Germany     107
...
```

In [ ]:
# soma de gols em casa por país
gols_casa = wc.groupby('time_1')['gols_1'].sum().rename('gols_casa')
# soma de gols fora por país (gols_2 correspondem aos gols do time_2 como visitante)
gols_fora = wc.groupby('time_2')['gols_2'].sum().rename('gols_fora')

# combinar
gols_total = pd.concat([gols_casa, gols_fora], axis=1).fillna(0)
gols_total['total_gols'] = gols_total['gols_casa'].astype(int) + gols_total['gols_fora'].astype(int)

# transformar em dataframe com coluna 'país'
gols = gols_total[['total_gols']].reset_index().rename(columns={'time_1':'país', 'time_2':'país'})
gols = gols.rename(columns={'index':'país'}) if 'index' in gols.columns else gols
gols = gols.sort_values('total_gols', ascending=False).reset_index(drop=True)

print("=== Total de gols por país (ordem decrescente) ===")
display(gols.head(20))

#### Q5. 
Qual país tomou mais cartões amarelos somando todas as copas?

Neste exercício você vai trabalhar com as colunas `cartao_amarelo_1` e `cartao_amarelo_2` onde cada observação é uma string represetando uma lista dos cartões amarelos de um único jogo na forma `minuto|placar|jogador(a)`. Por exemplo:
```
['16’|1:0|Rosana Gómez', '20’|2:0|Gabriela Chávez]
```

Recomendo criar colunas `num_cartoes_1` e `num_cartoes_2` a partir de cada coluna `cartao_amarelo_X` usando o método genérico `apply` para chamar uma função que você vai criar para transformar uma observação de cartão amarelo em uma contagem de elementos da lista. 

Lembre de levar em consideração que muitas observações são `NaN`. 

Em seguida faça sua mágica para agrupar as informações por país, acumular os cartões de jogos em casa e jogos visitante, e produzir o resultado final como apresentado a seguir (valores inventados):

```
país        cartões amarelos
Argentina   97
England     93
Australia   93
...
```

In [ ]:
# Função robusta para contar cartões em uma célula que pode ser:
# - NaN
# - uma representação de lista: "['16’|1:0|Nome', '20’|2:0|Nome']"
# - uma string com entradas separadas por '|' ou por "','"
import re

def contar_cartoes_celula(texto):
    if pd.isna(texto):
        return 0
    s = str(texto).strip()
    # Normaliza aspas tipográficas para aspas simples
    s = s.replace("’","'").replace("‘","'")
    # Tenta avaliar como literal Python (lista)
    try:
        val = ast.literal_eval(s)
        if isinstance(val, list):
            return len(val)
        # se for str após avaliação por algum motivo, cair no fallback
    except Exception:
        pass
    # Fallback: separar por "'],", "'], '", " | '", ou por "'],"
    # uma heurística: conte quantas vezes aparece o padrão de minuto seguido de pipe: e.g. "20’|"
    # mas noi dataset nem sempre tem o mesmo apóstrofo — vamos contar ocorrências de " | " delimiting items?
    # Simples e eficaz: se houver "]," ou ",'", use split por ",'" ou "],"
    # Caso contrário conte ocorrências de '|' e estime número de cartões como (count('|') // 2)
    # última opção: separa por ",'" ou '],' ou '| ' na tentativa de recuperar itens
    # Tentar splits ordenados:
    splits = None
    if "'], '" in s:
        splits = s.split("'], '")
    elif "']," in s:
        splits = s.split("'],")
    elif "', '" in s:
        splits = s.split("', '")
    elif ",' " in s:
        splits = s.split(",' ")
    else:
        # fallback por barras verticais: cada cartão tem pelo menos 2 pipes '|', então estimamos
        n_pipes = s.count("|")
        if n_pipes >= 2:
            # assume cada cartão possui ao menos 2 pipes (min|placar|jogador)
            return n_pipes // 2
        # se não conseguimos, retornar 1 como mínimo (texto não vazio)
        return 1 if len(s)>0 else 0

    # se fizemos split por algum padrão, contar itens válidos (removendo vazios)
    if splits is not None:
        itens = [it.strip() for it in re.split(r"\s*[,;]\s*", s) if it.strip()]
        # filtrar elementos que contenham dígitos de minuto ou pipe
        itens = [it for it in itens if '|' in it or re.search(r"\d+", it)]
        return len(itens)

    return 0

# criar colunas num_cartoes_1 e num_cartoes_2
wc['num_cartoes_1'] = wc['cartao_amarelo_1'].apply(contar_cartoes_celula) if 'cartao_amarelo_1' in wc.columns else 0
wc['num_cartoes_2'] = wc['cartao_amarelo_2'].apply(contar_cartoes_celula) if 'cartao_amarelo_2' in wc.columns else 0

# agrupar por país (casa e fora) e somar
cartoes_casa = wc.groupby('time_1')['num_cartoes_1'].sum()
cartoes_fora = wc.groupby('time_2')['num_cartoes_2'].sum()

cartoes_total = cartoes_casa.add(cartoes_fora, fill_value=0).sort_values(ascending=False)

# transformar em dataframe e exibir top 10
df_cartoes = cartoes_total.reset_index().rename(columns={'time_1':'país', 0:'cartões amarelos'})
df_cartoes.columns = ['país', 'cartões_amarelos']
print("=== Países por número total de cartões amarelos (decrescente) ===")
display(df_cartoes.head(20))

# país com mais amarelos
pais_mais_amarelos = df_cartoes.iloc[0]
print(f"\nPaís com mais cartões amarelos: {pais_mais_amarelos['país']} ({int(pais_mais_amarelos['cartões_amarelos'])} amarelos)")

#### Q6.
Qual o top10 jogadores com mais gols em copa? Considere gols em jogo e gols de pênalti.

Para conseguir essa informação, você precisará trabalhar com as colunas:
```
10  gols_1_detalhes         
11  gols_2_detalhes    
14  gols_1_penalti     
15  gols_2_penalti     
```

Essas também são colunas textuais, onde cada observações apresenta todos os gols de uma partida separados pelo caracter `|`. Cada gol está na forma `'Jogador(a) · minuto’'`. Por exemplo:
```
'Alex Morgan · 12’|Rose Lavelle · 20’'
```

Lembre de levar em consideração que muitas observações são `NaN`. 

Recomendo criar um dicionário à parte, onde cada chave será um jogador encontrado nessas colunas do dataframe `wc` e o valor correspondente será a contagem de ocorrências desses nomes.

Em seguida basta converter o seu dicionário em um dataframe e imprimí-lo na forma (valores não são inventados 😁):
```
jogador(a)      num_gols 
Marta           17
Miroslav Klose  16
... 
```

In [ ]:
import pandas as pd

# Carregar CSVs masculino e feminino
wcwomen_df = pd.read_csv('matches_1991_2023.csv')
wcmen_df   = pd.read_csv('matches_1930_2022.csv')

# Função para contar gols em um dataframe
def contar_gols_wc(df):
    gols_por_jogador = {}
    colunas_gols = ['home_goal', 'away_goal', 'home_penalty_goal', 'away_penalty_goal']
    
    for col in colunas_gols:
        if col not in df.columns:
            continue
        for linha in df[col].dropna():
            gols = linha.split('|')
            for gol in gols:
                jogador = gol.split('·')[0].strip().replace('\xa0','')
                if jogador:
                    gols_por_jogador[jogador] = gols_por_jogador.get(jogador,0) + 1
    return gols_por_jogador

# Conta gols masculino e feminino
gols_men = contar_gols_wc(wcmen_df)
gols_women = contar_gols_wc(wcwomen_df)

# Junta os dois dicionários
from collections import Counter
total_gols = Counter(gols_men) + Counter(gols_women)

# Converte em DataFrame
df_gols = pd.DataFrame(total_gols.items(), columns=['jogador(a)','num_gols'])
df_gols = df_gols.sort_values(by='num_gols', ascending=False).reset_index(drop=True)

# Top 10 absoluto (homens + mulheres)
top10_goleadores = df_gols.head(10)
display(top10_goleadores)